In [1]:
import os
import re
import logging
import datetime
import json
import csv
from statistics import fmean
from typing import Dict, List, Callable, Union
from graph_of_thoughts import controller, language_models, operations, prompter, parser
from graph_of_thoughts.vector_db import azure_embedding



# This is a hack to also allow execution of this file from the examples directory
try:
    from . import utils
except ImportError:
    import utils

class ALZKBPrompter(prompter.Prompter):
    """
    ALZKBPrompter provides the generation of prompts specific to the
    ALZKB example for the language models.

    Inherits from the Prompter class and implements its abstract methods.
    """
    
    alzkb_prompt_start = """You have access to a knowledge graph of Alzheimer's disease. The knowledge graph contains node types: Gene, DrugClass, Drug, Disease, Pathway, BiologicalProcess, MolecularFunction, CellularComponent, Symptom, BodyPart.
    The knowledge graph contains relationships: "CHEMICAL BINDS GENE","CHEMICAL INCREASES EXPRESSION", "CHEMICAL DECREASES EXPRESSION", "DRUG IN CLASS", "DRUG TREATS DISEASE", "DRUG CAUSES EFFECT", "GENE PARTICIPATES IN BIOLOGICAL PROCESS", "GENE IN PATHWAY", "GENE INTERACTS WITH GENE", "GENE HAS MOLECULAR FUNCTION", "GENE ASSOCIATED WITH CELLULAR COMPONENT", "GENE ASSOCIATES WITH DISEASE", "BODYPART OVER EXPRESSES GENE", "BODYPART UNDEREXPRESSES GENE", "SYMPTOM MANIFESTATION OF DISEASE", "DISEASE LOCALIZES TO ANATOMY", "DISEASE ASSOCIATES WITH DISEASE".
    """
    
    io_prompt = """<Instruction> You are an Alzheimer's data specialist AI assistant dedicated to providing information and support related to Alzheimer's disease.
Your primary goal is to assist users by offering factual and relevant information based on your access to a comprehensive knowledge graph associated with Alzheimer's. 
Your responses are focused on addressing queries related to Alzheimer's, and you do not provide information unrelated to the topic. 
You will also only answer based on the knowledge within the knowledge graph within the <Knowledge> tags. 
You will notice there will be gene symbols in the knowledge, and there are subtle differences between the gene names.
You will need to be careful that the names are exact with you use them in context. There may be single differences in numbers and letters.
For example, the gene "APOE" is not the same as gene "APOE1". Another example is the gene "IQCK" is not the same as gene "IQCG".
You will need to be careful of specific biological terms. For example, the term "amino" is different from the term "amine".
If you are providing a list, be sure not to list duplicates. 
Your demeanor is empathetic and concise as you aim to help users understand and navigate Alzheimer's-related concerns.
You will be provided knowledge within the <Knowledge> tags and must answer the question in the <Question> tags.
{additional_instruction}
</Instruction> 
<Knowledge>
{knowledge}
</Knowledge>
<Question>{question}</Question>"""
        
    cot_prompt = """<Instruction> You are an Alzheimer's data specialist AI assistant dedicated to providing information and support related to Alzheimer's disease.
Your primary goal is to assist users by offering factual and relevant information based on your access to a comprehensive knowledge graph associated with Alzheimer's. 
Your responses are focused on addressing queries related to Alzheimer's, and you do not provide information unrelated to the topic. 
You will also only answer based on the knowledge within the knowledge graph. 
You will notice there will be gene symbols in the knowledge, and there are subtle differences between the gene names.
You will need to be careful that the names are exact with you use them in context. There may be single differences in numbers and letters.
For example, the gene "APOE" is not the same as gene "APOE1". Another example is the gene "IQCK" is not the same as gene "IQCG".
You will need to be careful of specific biological terms. For example, the term "amino" is different from the term "amine".
If you are providing a list, be sure not to list duplicates. 
Your demeanor is empathetic and concise as you aim to help users understand and navigate Alzheimer's-related concerns.
You will be provided knowledge in the Knowledge section and must answer the question in the Question section. </Instruction> 
<Knowledge>
{knowledge}
</Knowledge>
<Question>{question}</Question>
<Approach>
To answer the question, follow these steps:
1. Identify the relevant nodes in the knowledge graph.
2. Identify the relevant relationships between the nodes.
3. Generate a response based on the identified nodes and relationships.
</Approach>

<Examples>
Input:

Output: 
</Examples>
"""

    
    score_prompt_base = """We are comparing if a response is equivalent to the ground truth.
<GroundTruth>{ground_truth}</GroundTruth>
<Answer>{answer}</Answer>
Please score the response on a scale of 0 to 10, where 0 is not equivalent and 10 is equivalent.
You may have to compare two unordered lists, so please score the response based on the following criteria:
1. The lists contain the same number of elements.
2. The lists contain the same elements, but may be in a different order.
If you have a True or False question, please score the response as 10 if the response is equal to the ground truth and 0 if the response is inequal.
You may provide reasoning for your scoring, but the final score should be between the tags <Score> and </Score>, without any additional text within those tags.
"""


    def __init__(self,vector_db, lm) -> None:
        """
        Inits the prompter.
        """
        self.vector_db = vector_db
        self.lm = lm
        pass
    
    def aggregation_prompt(self, state_dicts: List[Dict], **kwargs) -> str:
        """
        Generate an aggregation prompt for the language model.

        :param state_dicts: The thought states that should be aggregated.
        :type state_dicts: List[Dict]
        :param kwargs: Additional keyword arguments.
        :return: The aggregation prompt.
        :rtype: str
        :raise AssertionError: If not exactly two thought states are provided.
        """
        assert len(state_dicts) == 2, "Expected two states for aggregation prompt."

        return self.io_prompt.format(
            input1=state_dicts[0]["current"],
            input2=state_dicts[1]["current"],
        )

        if len(state_dicts[0]["parts"]) > 0 and len(state_dicts[0]["parts"]) < len(
            state_dicts[0]["documents"]
        ):
            prompt = self.aggregate_sub_prompt_base.format(
                num_ndas=len(state_dicts),
            )
            for i, state_dict in enumerate(state_dicts):
                prompt += self.aggregate_sub_prompt_generate.format(
                    nda=state_dict["current"], num=i + 1
                )
            return prompt
        else:
            prompt = self.aggregate_full_prompt_base.format(
                num_ndas=len(state_dicts[0]["documents"]),
                num_ndas_summary=len(state_dicts),
            )
            for i, document in enumerate(state_dicts[0]["documents"]):
                prompt += self.aggregate_full_prompt_block1.format(
                    document=document, num=i + 1
                )
            prompt += self.aggregate_full_prompt_mid.format(
                num_ndas_summary=len(state_dicts),
            )
            for i, state_dict in enumerate(state_dicts):
                prompt += self.aggregate_full_prompt_block2.format(
                    summary=state_dict["current"], num=i + 1
                )
            return prompt

    def generate_prompt(
        self,
        num_branches: int,
        question: str,
        question_type: str,
        method: str,
        current: str,
        **kwargs,
    ) -> str:
        """
        Generate a generate prompt for the language model.

        :param num_branches: The number of responses the prompt should ask the LM to generate.
        :type num_branches: int
        :param documents: The list of documents to be merged.
        :type documents: List[str]
        :param method: Method for which the generate prompt is generated.
        :type method: str
        :param parts: Indices of the already processed document parts.
        :type parts: Set[str]
        :param current: The intermediate solution.
        :type current: str
        :param kwargs: Additional keyword arguments.
        :return: The generate prompt.
        :rtype: str
        :raise AssertionError: If method is not implemented yet.
        """
        

        if question_type == "true/false":
            additional_instruction = "You will be asked to answer the question with only a TRUE or FALSE response."
            # check if "True or False Question: " is in the question
            if "True or False Question: " in question:
                statement_to_embed = question[question.index("True or False Question: ") + len("True or False Question: "):]
            else:
                statement_to_embed = question
                question = "True or False Question: " + question
        elif question_type == "multiple choice":
            additional_instruction = "You will be asked to answer the question with only the multiple choice number response. For instance, if the correct answer is '2', you will need to answer '2'."
            # check if "? 1." is in the question and separate the question from the choice
            if "? 1." in question:
                statement_to_embed = question[:question.index("? 1.")]
            else:
                statement_to_embed = question
        elif question_type == "list":
            additional_instruction = "You will be asked to answer the question with only the list with each element separated by a newline."
            statement_to_embed = question
        else:
            additional_instruction = ""
            statement_to_embed = question
            
        # print("statement_to_embed:", statement_to_embed)
        # print("question:", question)
        embedded_question = self.lm.get_embedding(statement_to_embed)
        # print(embedded_question)
        knowledge_array,distances = self.vector_db.get_knowledge(embedded_question)
        # print(knowledge_array)
        knowledge = "\n".join(knowledge_array)
        
        prompt = ""
        assert num_branches == 1, "Branching should be done via multiple requests."
        if method.startswith("io"):
            # print(self.io_prompt.format(knowledge=knowledge, question=question, additional_instruction=additional_instruction))
            return self.io_prompt.format(knowledge=knowledge, question=question, additional_instruction=additional_instruction)
        elif method.startswith("cot"):
            prompt += self.alzkb_prompt_start + self.cot_prompt.format(knowledge=knowledge, question=question, additional_instruction=additional_instruction)
            return prompt
        # elif method.startswith("cot"):
        #     return self.intersection_prompt_cot.format(set1=set1, set2=set2)
        # elif method.startswith("tot"):
        #     if current is None or current == "":
        #         return self.intersection_prompt.format(set1=set1, set2=set2)
        #     return self.tot_improve_prompt.format(
        #         set1=set1, set2=set2, incorrect_intersection=current
        #     )
        # elif method.startswith("got"):
        #     if kwargs["phase"] == 0:
        #         return self.got_split_prompt.format(input=set2)

        #     input_set = set2
        #     if "subset" in kwargs and kwargs["subset"] != "":
        #         input_set = kwargs["subset"]

        #     return self.intersection_prompt.format(set1=set1, set2=input_set)
        
        
        # if method.startswith("io") or method.startswith("cot"):
        #     if method.startswith("io"):
        #         prompt += self.merge_doc_prompt_start.format(num=len(documents))
        #     else:
        #         prompt += self.merge_doc_prompt_cot_start.format(num=len(documents))
        #     for i, document in enumerate(documents):
        #         prompt += self.merge_doc_prompt_block.format(
        #             document=document, num=i + 1
        #         )
        #     return prompt
        # elif method.startswith("tot"):
        #     if current is None or current == "":
        #         prompt += self.merge_doc_prompt_start.format(num=len(documents))
        #         for i, document in enumerate(documents):
        #             prompt += self.merge_doc_prompt_block.format(
        #                 document=document, num=i + 1
        #             )
        #         return prompt
        #     else:
        #         prompt += self.improve_summary_prompt_start.format(
        #             num=len(documents),
        #         )
        #         for i, document in enumerate(documents):
        #             prompt += self.improve_summary_prompt_block.format(
        #                 document=document, num=i + 1
        #             )
        #         prompt += self.improve_summary_prompt_end.format(summary=current)
        #         return prompt
        # elif method.startswith("got"):
        #     parts = (
        #         sorted(list(parts)) if len(parts) > 0 else list(range(len(documents)))
        #     )
        #     if current is None or current == "":
        #         prompt += self.merge_doc_prompt_start.format(num=len(parts))
        #         for i, part in enumerate(sorted(list(parts))):
        #             prompt += self.merge_doc_prompt_block.format(
        #                 document=documents[part], num=i + 1
        #             )
        #         return prompt
        #     else:
        #         prompt += self.improve_summary_prompt_start.format(
        #             num=len(parts),
        #         )
        #         for i, part in enumerate(sorted(list(parts))):
        #             prompt += self.improve_summary_prompt_block.format(
        #                 document=documents[part], num=i + 1
        #             )
        #         prompt += self.improve_summary_prompt_end.format(summary=current)
        #         return prompt
        else:
            assert False, "Not implemented yet."

    def improve_prompt(self, **kwargs) -> str:
        """
        Generate an improve prompt for the language model.

        :param kwargs: Additional keyword arguments.
        :return: The improve prompt.
        :rtype: str
        """
        pass

    def validation_prompt(self, **kwargs) -> str:
        """
        Generate a validation prompt for the language model.

        :param kwargs: Additional keyword arguments.
        :return: The validation prompt.
        :rtype: str
        """
        pass

    def score_prompt(self, state_dicts: List[Dict], **kwargs) -> str:
        """
        Generate a score prompt for the language model.

        :param state_dicts: The thought states that should be scored,
                            if more than one, they should be scored together.
        :type state_dicts: List[Dict]
        :param kwargs: Additional keyword arguments.
        :return: The score prompt.
        :rtype: str
        :raise AssertionError: If more than one thought state is supplied.
        """

        # perform individual scoring
        prompt = self.score_prompt_base.format(
            answer=state_dicts[0]["current"],
            ground_truth=state_dicts[0]["ground_truth"],
        )
        return prompt
    
class ALZKBParser(parser.Parser):
    """
    ALZKBParser provides the parsing of language model reponses specific to the
    ALZKB example.

    Inherits from the Parser class and implements its abstract methods.
    """

    def __init__(self) -> None:
        """
        Inits the response cache.
        """
        self.cache = {}
        
    def strip_answer_helper(self, text: str, tag: str = "") -> str:
        """
        Helper function to remove tags from a text.

        :param text: The input text.
        :type text: str
        :param tag: The tag to be stripped. Defaults to "".
        :type tag: str
        :return: The stripped text.
        :rtype: str
        """

        text = text.strip()
        if "Output:" in text:
            text = text[text.index("Output:") + len("Output:") :].strip()
        if tag != "":
            start = text.rfind(f"<{tag}>")
            end = text.rfind(f"</{tag}>")
            if start != -1 and end != -1:
                text = text[start + len(f"<{tag}>") : end].strip()
            elif start != -1:
                logging.warning(
                    f"Only found the start tag <{tag}> in answer: {text}. Returning everything after the tag."
                )
                text = text[start + len(f"<{tag}>") :].strip()
            elif end != -1:
                logging.warning(
                    f"Only found the end tag </{tag}> in answer: {text}. Returning everything before the tag."
                )
                text = text[:end].strip()
            else:
                logging.warning(
                    f"Could not find any tag {tag} in answer: {text}. Returning the full answer."
                )
        return text

    def parse_aggregation_answer(
        self, states: List[Dict], texts: List[str]
    ) -> Union[Dict, List[Dict]]:
        """
        Parse the response from the language model for an aggregation prompt.

        :param states: The thought states used to generate the prompt.
        :type states: List[Dict]
        :param texts: The responses to the prompt from the language model.
        :type texts: List[str]
        :return: The new thought states after parsing the respones from the language model.
        :rtype: Union[Dict, List[Dict]]
        :raise AssertionError: If not exactly two thought states are provided.
        """

        assert len(states) == 2, "Expected two states for aggregation answer."
        new_states = []
        for text in texts:
            answers = text.strip().split("\n")
            if any(["Output" in answer for answer in answers]):
                # cut elements until last output is found
                for answer in reversed(answers):
                    if "Output" in answer:
                        answers = answers[answers.index(answer) :]
                        break

            answers_stripped = [
                answer for answer in answers if "[" in answer and "]" in answer
            ]
            if len(answers_stripped) == 0:
                for answer in answers:
                    answer = "[" + answer + "]"
                    try:
                        answer_converted = utils.string_to_list(answer)
                        if len(answer_converted) > 0:
                            answers_stripped.append(answer)
                    except:
                        pass
            if len(answers_stripped) == 0:
                logging.warning(
                    f"Could not parse aggregation answer: {text}. Returning empty list."
                )
                answer = "[]"
            else:
                answer = [
                    answer[answer.index("[") : answer.index("]") + 1]
                    for answer in answers_stripped
                ][0]
            states = sorted(states, key=lambda x: x["part"])
            merged_subsets = states[0]["subset"][:-1] + ", " + states[1]["subset"][1:]
            new_state = states[0].copy()
            new_state["current"] = answer
            new_state["subset"] = merged_subsets
            new_states.append(new_state)
        return new_states

    def parse_improve_answer(self, state: Dict, texts: List[str]) -> Dict:
        """
        Parse the response from the language model for an improve prompt.

        :param state: The thought state used to generate the prompt.
        :type state: Dict
        :param texts: The responses to the prompt from the language model.
        :type texts: List[str]
        :return: The new thought state after parsing the responses from the language model.
        :rtype: Dict
        """
        pass

    def parse_generate_answer(self, state: Dict, texts: List[str]) -> List[Dict]:
        """
        Parse the response from the language model for a generate prompt.

        :param state: The thought state used to generate the prompt.
        :type state: Dict
        :param texts: The responses to the prompt from the language model.
        :type texts: List[str]
        :return: The new thought states after parsing the respones from the language model.
        :rtype: List[Dict]
        """

        new_states = []
        print("parse_generate_answer text:", texts)
        
        for text in texts:
            if state["method"].startswith("io") or state["method"].startswith("cot"):
                try:
                    if state["question_type"] == "true/false":
                        if "TRUE" in text.upper():
                            answer = "TRUE"
                        elif "FALSE" in text.upper():
                            answer = "FALSE"
                        else:
                            answer = "FALSE"
                    elif state["question_type"] == "multiple choice":
                        #white strip to remove any spaces and check for numbers in the first couple of characters
                        if any(char.isdigit() for char in text[:3]):
                            answer = text[0]
                        else:
                            logging.warning(
                                f"Could not parse step answer: {text}. Returning empty."
                            )
                            continue
                    elif state["question_type"] == "list":
                        temp_answer = text.strip().split("\n")
                        answer = [x for x in temp_answer if x != '']
                        #check for empty answers
                        if len(answer) == 0:
                            logging.warning(
                                f"Could not parse step answer: {text}. Returning empty."
                            )
                            continue
                        
                    else:
                        answer = text.strip()
                        
                        
                    new_state = state.copy()
                    new_state["current"] = answer
                    new_state["phase"] = new_state["phase"] + 1
                    new_states.append(new_state)
                except Exception as e:
                    logging.error(
                        f"Could not parse step answer: {text}. Encountered exception: {e}"
                    )
                        
                        
            elif state["method"].startswith("got") and state["phase"] == 0:
                # We expect a json which contains the two lists named "List 1" and "List 2"
                # cut everything until the opening bracket and everything after the closing bracket

                try:
                    text = text[text.index("{") : text.index("}") + 1]
                    json_dict = json.loads(text)
                    if len(json_dict.keys()) != 2:
                        logging.warning(
                            f"Expected 2 lists in json, but found {len(json_dict.keys())}."
                        )
                    for key, value in json_dict.items():
                        if "List" not in key:
                            logging.warning(
                                f"Expected key to contain 'List', but found {key}."
                            )
                            continue
                        if not isinstance(value, list):
                            value = utils.string_to_list(value)
                        new_state = state.copy()
                        new_state["current"] = ""
                        new_state["subset"] = str(value)
                        new_state["phase"] = 1
                        new_state["part"] = key
                        new_states.append(new_state)
                except Exception as e:
                    logging.error(
                        f"Could not parse step answer: {text}. Encountered exception: {e}"
                    )
            else:
                answers = text.strip().split("\n")
                answers = [
                    answer for answer in answers if "[" in answer and "]" in answer
                ]
                if any(["Output" in answer for answer in answers]):
                    # cut elements until last output is found
                    for answer in reversed(answers):
                        if "Output" in answer:
                            answers = answers[answers.index(answer) :]
                            break

                answers = [
                    answer[answer.index("[") : answer.index("]") + 1]
                    for answer in answers
                ]
                if len(answers) == 0:
                    logging.warning(
                        f"Could not parse step answer: {text}. Returning empty list."
                    )
                    answer = "[]"
                else:
                    if len(answers) > 1:
                        logging.warning(
                            f"Multiple answers found for step answer: {text}. Using the first one."
                        )
                    answer = answers[0]

                new_state = state.copy()
                new_state["current"] = answer
                new_state["phase"] = 2
                new_states.append(new_state)
        return new_states

    def parse_validation_answer(self, state: Dict, texts: List[str]) -> bool:
        """
        Parse the response from the language model for a validation prompt.

        :param state: The thought state used to generate the prompt.
        :type state: Dict
        :param texts: The responses to the prompt from the language model.
        :type texts: List[str]
        :return: Whether the thought state is valid or not.
        :rtype: bool
        """
        pass

    def parse_score_answer(self, states: List[Dict], texts: List[str]) -> List[float]:
        """
        Parse the response from the language model for a score prompt.

        :param states: The thought states used to generate the prompt.
        :type states: List[Dict]
        :param texts: The responses to the prompt from the language model.
        :type texts: List[str]
        :return: The scores for the thought states.
        :rtype: List[float]
        """
        assert len(states) == 1, "Only one state is allowed for scoring."
        if len(states) == 1:
            # individual scoring
            redundancy_scores = []
            retain_scores = []
            for text in texts:
                answer = self.strip_answer_helper(text, "Score")
                res = re.findall(r"\d+\.?\d*", answer)
                if len(res) == 1:
                    redundancy_scores.append(float(res[0]))
                elif len(res) > 1:
                    logging.warning(
                        f"Found multiple scores in answer: {text}. Returning the last one."
                    )
                    redundancy_scores.append(float(res[-1]))
                else:
                    logging.warning(
                        f"Could not find any score in answer: {text}. Ignoring this answer."
                    )
            if len(redundancy_scores) == 0:
                logging.warning(
                    f"Could not find any valid score in any answer. Returning 0.0."
                )
                return [0.0]
            mean_redundancy = fmean(redundancy_scores)
            print("mean_score:", mean_redundancy)
            return [mean_redundancy]
            # mean_retain = fmean(retain_scores)
            # f1 = 2 * mean_redundancy * mean_retain / (mean_redundancy + mean_retain)
            # return [f1]


def io() -> operations.GraphOfOperations:
    """
    Generates the Graph of Operations for the IO method.

    :return: Graph of Operations
    :rtype: GraphOfOperations
    """
    operations_graph = operations.GraphOfOperations()

    operations_graph.append_operation(operations.Generate(1, 1))
    operations_graph.append_operation(operations.Score(1, False))
    # operations_graph.append_operation(operations.GroundTruth())

    return operations_graph


def cot() -> operations.GraphOfOperations:
    """
    Generates the Graph of Operations for the CoT method.

    :return: Graph of Operations
    :rtype: GraphOfOperations
    """
    operations_graph = operations.GraphOfOperations()

    operations_graph.append_operation(operations.Generate(1, 1))
    operations_graph.append_operation(operations.Score(1, False))

    return operations_graph


def tot() -> operations.GraphOfOperations:
    """
    Generates the Graph of Operations for the ToT method.

    :return: Graph of Operations
    :rtype: GraphOfOperations
    """
    operations_graph = operations.GraphOfOperations()

    branch_factor = 10

    operations_graph.append_operation(operations.Generate(1, branch_factor))
    operations_graph.append_operation(operations.Score(3, False))
    keep_best_1 = operations.KeepBestN(1, True)
    operations_graph.append_operation(keep_best_1)

    for _ in range(2):
        operations_graph.append_operation(operations.Generate(1, branch_factor))
        operations_graph.append_operation(operations.Score(3, False))
        keep_best_2 = operations.KeepBestN(1, True)
        keep_best_2.add_predecessor(keep_best_1)
        operations_graph.append_operation(keep_best_2)
        keep_best_1 = keep_best_2

    return operations_graph


def got() -> operations.GraphOfOperations:
    """
    Generates the Graph of Operations for the GoT method, where full documents
    are merged.

    :return: Graph of Operations
    :rtype: GraphOfOperations
    """
    operations_graph = operations.GraphOfOperations()

    operations_graph.append_operation(operations.Generate(1, 5))
    operations_graph.append_operation(operations.Score(3, False))
    keep_best = operations.KeepBestN(3, True)
    operations_graph.append_operation(keep_best)
    operations_graph.append_operation(operations.Aggregate(5))
    operations_graph.append_operation(operations.Score(3, False))
    keep_best2 = operations.KeepBestN(1, True)
    keep_best2.add_predecessor(keep_best)
    operations_graph.append_operation(keep_best2)
    operations_graph.append_operation(operations.Generate(1, 10))
    operations_graph.append_operation(operations.Score(3, False))
    keep_best3 = operations.KeepBestN(1, True)
    keep_best3.add_predecessor(keep_best2)
    operations_graph.append_operation(keep_best3)

    return operations_graph


def got2() -> operations.GraphOfOperations:
    """
    Generates the Graph of Operations for the GoT2 method, where partial
    documents are merged.

    :return: Graph of Operations
    :rtype: GraphOfOperations
    """
    operations_graph = operations.GraphOfOperations()

    sub_parts = []
    for i in range(0, 4, 2):  # should be at most 16 parts
        sub_text = operations.Selector(
            lambda thoughts, list_id=i: [
                operations.Thought(
                    state={**thoughts[0].state, "parts": {list_id, list_id + 1}}
                )
            ]
        )
        operations_graph.add_operation(sub_text)
        gen_nda = operations.Generate(1, 5)
        gen_nda.add_predecessor(sub_text)
        operations_graph.add_operation(gen_nda)
        score_nda = operations.Score(3, False)
        score_nda.add_predecessor(gen_nda)
        operations_graph.add_operation(score_nda)
        keep_best_nda = operations.KeepBestN(1, True)
        keep_best_nda.add_predecessor(score_nda)
        operations_graph.add_operation(keep_best_nda)

        sub_parts.append(keep_best_nda)

    while len(sub_parts) > 1:
        new_sub_parts = []
        for i in range(0, len(sub_parts), 2):
            if i + 1 == len(sub_parts):
                new_sub_parts.append(sub_parts[i])
                continue
            aggregate = operations.Aggregate(5)
            aggregate.add_predecessor(sub_parts[i])
            aggregate.add_predecessor(sub_parts[i + 1])
            operations_graph.add_operation(aggregate)
            score = operations.Score(3, False)
            score.add_predecessor(aggregate)
            operations_graph.add_operation(score)
            keep_best = operations.KeepBestN(1, True)
            keep_best.add_predecessor(score)
            operations_graph.add_operation(keep_best)

            gen_nda = operations.Generate(1, 5)
            gen_nda.add_predecessor(keep_best)
            operations_graph.add_operation(gen_nda)
            score_nda = operations.Score(3, False)
            score_nda.add_predecessor(gen_nda)
            operations_graph.add_operation(score_nda)
            keep_best_nda = operations.KeepBestN(1, True)
            keep_best_nda.add_predecessor(score_nda)
            keep_best_nda.add_predecessor(keep_best)
            operations_graph.add_operation(keep_best_nda)

            new_sub_parts.append(keep_best_nda)
        sub_parts = new_sub_parts

    return operations_graph




C:\Users\matsumoton\AppData\Local\Temp\ipykernel_22256\1607591916.py:16: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  from . import utils


In [8]:
#get json files in directory
import json
import os
directory = r'C:/Users/matsumoton/Box/CAIRE Work Folder/AlzKB – Neo4J and LLMs Project/AlzKB_LLM_Evaluation'
questions = {}
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        with open(os.path.join(directory, filename)) as f:
            questions[filename] = json.load(f)

In [10]:
import dill
dill.dump(questions, open("questions.pkl", "wb"))


C:\Users\matsumoton\AppData\Local\Temp\ipykernel_27432\2676934179.py:2: ResourceWarning: unclosed file <_io.BufferedWriter name='questions.pkl'>
  dill.dump(questions, open("questions.pkl", "wb"))


In [4]:
questions

{'MCQ_1hop.json': [{'question': 'Which of the following binds to the drug Leucovorin? 1. CAD 2. PDS5B 3. SEL1L 4. ABCC2 5. RMI1',
   'answer': '4'},
  {'question': 'Which of the following binds to the drug Chlormerodrin? 1. PDS5A 2. RMI1 3. CAD 4. PDS5B 5. SLC12A1',
   'answer': '5'},
  {'question': 'Which of the following binds to the drug Papaverine? 1. SEL1L 2. CAD 3. PDS5B 4. PDS5A 5. PDE4B',
   'answer': '5'},
  {'question': 'Which of the following binds to the drug Ethchlorvynol? 1. RMI1 2. GABRB3 3. SEL1L 4. PDS5B 5. CAD',
   'answer': '2'},
  {'question': 'Which of the following binds to the drug Methimazole? 1. PDS5B 2. PDS5A 3. CYP3A4 4. RMI1 5. SEL1L',
   'answer': '3'},
  {'question': 'Which of the following binds to the drug Amoxapine? 1. RMI1 2. SEL1L 3. CAD 4. PDS5A 5. HTR1A',
   'answer': '5'},
  {'question': 'Which of the following binds to the drug Amobarbital? 1. PDS5B 2. RMI1 3. GABRA5 4. PDS5A 5. CAD',
   'answer': '3'},
  {'question': 'Which of the following binds

In [8]:
import dill

dill.dump(operations_log, open("operations_log_io.pkl", "wb"))

C:\Users\matsumoton\AppData\Local\Temp\ipykernel_22256\4217967991.py:3: ResourceWarning: unclosed file <_io.BufferedWriter name='operations_log_io.pkl'>
  dill.dump(operations_log, open("operations_log_io.pkl", "wb"))


In [11]:

output = []
for operation in operations_log['MCQ_1hop.json']['Which of the following binds to the drug Leucovorin? 1. CAD 2. PDS5B 3. SEL1L 4. ABCC2 5. RMI1']:
    operation_serialized = {
        "operation": operation.operation_type.name,
        "thoughts": [thought.state for thought in operation.get_thoughts()],
    }
    if any([thought.scored for thought in operation.get_thoughts()]):
        operation_serialized["scored"] = [
            thought.scored for thought in operation.get_thoughts()
        ]
        operation_serialized["scores"] = [
            thought.score for thought in operation.get_thoughts()
        ]
    if any([thought.validated for thought in operation.get_thoughts()]):
        operation_serialized["validated"] = [
            thought.validated for thought in operation.get_thoughts()
        ]
        operation_serialized["validity"] = [
            thought.valid for thought in operation.get_thoughts()
        ]
    if any(
        [
            thought.compared_to_ground_truth
            for thought in operation.get_thoughts()
        ]
    ):
        operation_serialized["compared_to_ground_truth"] = [
            thought.compared_to_ground_truth
            for thought in operation.get_thoughts()
        ]
        operation_serialized["problem_solved"] = [
            thought.solved for thought in operation.get_thoughts()
        ]
    output.append(operation_serialized)

In [13]:
question["question"]

'List the genes which are over-expressed in testis'

In [14]:
from graph_of_thoughts import controller, language_models, operations

from graph_of_thoughts.vector_db.weaviate import WeaviateClient
import logging
import dill
import time
with open("questions.pkl", "rb") as f:
    questions = dill.load(f)

# Problem input

# operations_log = {}
with open("operations_log_io.pkl", "rb") as f:
    operations_log = dill.load(f)
    


for json_file,values in questions.items():
    print(json_file)
    if json_file.startswith("MCQ"):
      question_type = "multiple choice"
    elif json_file.startswith("True_or_False"):
      question_type = "true/false"
    elif json_file.startswith("OpenEnded_1hop"):
      question_type = "list"
    elif json_file.startswith("OpenEnded_2hop"):
      question_type = "list"
    elif json_file.startswith("OpenEnded_genes"):
      question_type = "open ended"
    
    if json_file not in operations_log:
      operations_log[json_file] = {}
    for question in values:
        print('question:', question["question"])
        print('answer:', question["answer"])
        
        if question["question"] in operations_log[json_file]:
          # print("already done")
          continue

        def io() -> operations.GraphOfOperations:
          """
          Generates the Graph of Operations for the IO method.

          :return: Graph of Operations
          :rtype: GraphOfOperations
          """
          operations_graph = operations.GraphOfOperations()

          operations_graph.append_operation(operations.Generate(1, 1))
          operations_graph.append_operation(operations.Score(1, False))
          # operations_graph.append_operation(operations.GroundTruth())

          return operations_graph
        # Retrieve the Graph of Operations
        io = io()

        config_file = "config.json"
        config_file = "config_v1.json"

        # Configure the Language Model (Assumes config.json is in the current directory with OpenAI API key)
        lm = language_models.AzureGPT(config_file, model_name="azuregpt")
        vdb = WeaviateClient(config_file)
        
        # Create the Controller
        try:
          ctrl = controller.Controller(
            lm, 
            io, 
            ALZKBPrompter(vdb,lm), 
            ALZKBParser(),
            # The following dictionary is used to configure the initial thought state
            {
              "question": question["question"],
              "ground_truth" : question["answer"],
              "question_type": question_type,
              "current": "",
              "phase": 0,
              "method": "io"
              # "method" : "got"
            }
          )
          
          # ctrl.logger.setLevel(logging.DEBUG)

          # Run the Controller and generate the output graph
          ctrl.run()
        except Exception as e:
          print(e)
        # ctrl.output_graph("output_got.json")
        operations_log[json_file][question["question"]] = ctrl.graph.operations
        
        # delete the controller to free up memory
        del ctrl
        # time.sleep(1)
        #break
  #break
    



MCQ_1hop.json
question: Which of the following binds to the drug Leucovorin? 1. CAD 2. PDS5B 3. SEL1L 4. ABCC2 5. RMI1
answer: 4
question: Which of the following binds to the drug Chlormerodrin? 1. PDS5A 2. RMI1 3. CAD 4. PDS5B 5. SLC12A1
answer: 5
question: Which of the following binds to the drug Papaverine? 1. SEL1L 2. CAD 3. PDS5B 4. PDS5A 5. PDE4B
answer: 5
question: Which of the following binds to the drug Ethchlorvynol? 1. RMI1 2. GABRB3 3. SEL1L 4. PDS5B 5. CAD
answer: 2
question: Which of the following binds to the drug Methimazole? 1. PDS5B 2. PDS5A 3. CYP3A4 4. RMI1 5. SEL1L
answer: 3
question: Which of the following binds to the drug Amoxapine? 1. RMI1 2. SEL1L 3. CAD 4. PDS5A 5. HTR1A
answer: 5
question: Which of the following binds to the drug Amobarbital? 1. PDS5B 2. RMI1 3. GABRA5 4. PDS5A 5. CAD
answer: 3
question: Which of the following binds to the drug Doxazosin? 1. PDS5B 2. SEL1L 3. CAD 4. PDS5A 5. KCNH7
answer: 5
question: Which of the following binds to the drug 

INFO:backoff:Backing off chat(...) for 0.9s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 22927 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})
INFO:backoff:Backing off chat(...) for 0.0s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 22927 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})
INFO:backoff:Backing off chat(...) for 0.2s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 22927 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'message

Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 22927 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
question: List the genes which are over-expressed in nipple
answer: NUSAP1, KBTBD4, RBMS1, ADORA2B, KLHL28, GGA3, HLA-C, DCD, ARPC2, EPHX3, SPSB3, TSPAN14, ITGA11, ACAD9, USP39, TMTC3, FOXK2, CDON, ASCL2, PPIF, GNA11, MYLK, RBMX2, WFDC5, PTER, RAB11FIP1, CBX4, CISH, SLC25A5, ANKRD35, GNB4, POU2F2, FKBP14, DYNC2I2, SERPINH1, GPATCH2L, C5orf46, TSHZ3, THAP4, APOE, OGDH, LOXL1, MYEOV, SLC35E1, MICALL1, NFX1, TSPAN5, MAF1, PRELP, CPED1, ZBTB7B, NLRP1, TPM2, NSFL1C, SFXN3, ANO1, FBXO17, LRRC1, PNPLA3, MAP3K1, PEAK1, COL1A2, NIPAL4, EFEMP2, TRIB3, PLEKHA2, CSNK2A1, PWP1, PUF60, NABP2, KATNB1, COL6A2, BAZ2A, PM20D1, EIF4A3, CCL28, SLCO3A1, SKI, SINHCAF, C1QTNF12, TSPO, COL18A1, METTL16, TCF19, DHRS1, ATPAF2, NFAT5, CCDC5

INFO:backoff:Backing off chat(...) for 0.7s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, you requested 16468 tokens (15268 in the messages, 1200 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


parse_generate_answer text: ['GCN1\nGTF3C1\nIFT122\nFIG4\nGTF2F1\nPOT1\nGET1\nGTF3C5\nPTPRA\nVIPAS39\nPROX1\nIFT140\nGTF2F2\nIFT56\nIFT27\nC1orf56\nMAT2A\nGTF2A2\nPAIP1\nTPD52\nPPP2R3C\nC1orf115\nGPR158\nDNAH5\nIFT172\nHID1\nIFT22\nIFT80\nPABPC1L\nPIP5K1A\nPANK4\nC22orf15\nPPP1R16A\nHTRA1\nGLG1\nIFT88\nPRELID3B\nPER1\nPTPRN\nCEP104\nH1-10\nTCTN1\nIFT81\nC1orf53\nHAPLN1\nPPIP5K1\nPTP4A3\nPRR14\nSUPT16H\nPPP2R2D\nHARS1\nGPR50\nMR1\nGPR19\nDDX42\nSLC25A22\nGHRHR\nIFT52\nIFT57\nPTPRG\nIFT46\nPPIP5K2\nC1orf122\nCIB1\nSLC35G1\nVPS9D1\nTPRG1L\nSLC25A14\nGFPT1\nFGFR1\nSLC25A45\nUSP22\nNAT14\nCOPG1\nMRPL49\nPTN\nPTBP1\nPPP2R2C\nHMGN5\nNAP1L5\nFKBP14\nHM13\nGATAD2A\nSLC22A14\nTEX261\nSLC46A1\nP4HA2\nTRAPPC14\nGPR173\nH2BC21\nMRPL23\nPLAG1\nHIC2\nMAU2\nPLEKHG4B\nCITED1\nTPD52L2\nC1orf50\nSLC6A15\nPREPL\nN4BP2L1\nPTPRS\nPPP1R21\nPITX1\nDNAH7\nPOU2F1\nPTPN13\nSUPT5H\nACTR3B\nVARS1\nC14orf93\nTSC22D1\nSUPT6H\nCEP290\nSLC25A39\nP4HB\nLDOC1\nHINT1\nP3H4\nVGF\nACTL6B\nP3H1\nPPP2R2B\nCTTN\nCEP44\nACTR6\

...

Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 27822 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
question: List the genes which are under-expressed in adipose tissue
answer: CIBAR2, PDX1, SIRT2, SPINT2, KIAA0753, BRINP1, TIAM2, SLC4A8, TP53TG3, DLG3, ZNF292, SRRT, TRIM17, POP1, UBASH3A, UBA7, ATG4D, LEFTY1, SLC4A1AP, MICAL2, ST6GAL1, LHFPL1, CD247, NKRF, NPY2R, PRUNE1, HORMAD1, NEUROG2, DPF3, CBX3, CAMK2G, SERP2, RHOF, ADAM30, LIN28B, BCL11B, TMEM82, BRD8, STK17A, H2BC5, TM4SF19, ELAPOR1, OCM, SPACDR, PCDH17, MBP, YY1AP1, CAPN13, FKBP11, PLEKHA6, DCC, ATXN1, PTPRZ1, BTN3A3, P2RY10, FAM24B, DNER, SERPINA11, PAX5, STARD4, CXCR6, DHRS7, ZNF771, PCDH10, DOCK3, SMG8, MGME1, CD47, MTERF1, AMMECR1, SCAMP5, SACM1L, MFSD2A, KIAA2012, EDAR, PRPF19, SEMA4D, SSPOP, ZNF674, ST8SIA1, DEFB118, ZFYVE27, IL1B, HS3ST3B1, ARAP2

INFO:backoff:Backing off chat(...) for 0.3s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 26730 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


parse_generate_answer text: ['ADI1\nIFT122\nUNC119B\nPRADC1\nADAL\nHAP1\nUSP47\nIFT81\nIFT88\nUNC5D\nGPR153\nDEFB119\nREG1A\nIFT172\nTRIM29\nADPRHL1\nACTR1A\nAADAC\nGPR158\nADO\nGPR39\nGPR107\nLOC101929552\nADGRV1\nADPRM\nIFT22\nMRPL55\nMED25\nIFT140\nCASP9\nREG4\nGID8\nDNAH11\nICA1L\nIFT74\nING4\nCAB39\nFIG4\nUNC79\nATAD1\nEFL1\nADK\nMRPL35\nCREG1\nPROM1\nDNAH5\nUSP54\nGET1\nC1orf56\nLINC03040\nGPR87\nMRPL14\nUSP46\nC22orf39\nINPP5A\nLAD1\nACTL9\nERP44\nUSP37\nACTRT1\nMAT1A\nGADD45A\nACTA1\nH1-8\nUNC45B\nTIPRL\nC1orf216\nC1orf94\nTRAPPC11\nHABP4\nHMX2\nUSP53\nACTR3B\nGDNF\nDEFB129\nPRODH2\nGATC\nPROM2\nC1orf105\nMRPL36\nZNF415\nADCK2\nADIG\nCWC27\nACAD8\nPRODH\nHACD3\nHIGD1B\nMRPL44\nACTR8\nGPR108\nMRPL37\nACTC1\nDEFB105A\nDEFA5\nALAD\nTRIM55\nACADL\nCOL28A1\nHABP2\nHAT1\nUSP43\nAAGAB\nINPP5F\nZXDB\nTHEGL\nCA8\nATAD2\nH1-6\nABCC5\nHBP1\nINSIG1\nDEFB118\nCT47A8\nC1orf50\nHLF\nMRPL43\nGLG1\nDEFA4\nREG3A\nREG3G\nGCN1\nC1orf116\nPABPC1L2A\nGPRASP3\nCT47A7\nC1orf226\nC22orf31\nGID4\nUSP44\

INFO:backoff:Backing off chat(...) for 0.4s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 26730 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})
INFO:backoff:Backing off chat(...) for 2.0s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 26730 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})
INFO:backoff:Backing off chat(...) for 3.0s (openai.BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 26730 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'message

Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 26730 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
question: List the genes which are under-expressed in smooth muscle tissue
answer: CASC3, MAN1C1, RASSF2, RIDA, BEX1, GABRB1, B4GAT1, R3HDM2, CAMKK2, FCN3, SLC48A1, NIBAN1, ADA2, ACSL1, PEG10, FKBP4, NXF1, CDH1, CRH, VSIG4, PSG11, EPB41L1, CIT, NDRG1, STXBP1, SCAP, DNM3, WDR82, TMEM176B, SH3TC1, OGT, C7, S100A9, P2RY13, C5AR1, DSP, MAST4, AMD1, NPTXR, GLUL, CA4, PSG5, TBXAS1, PAK3, GABBR1, PAQR6, SCG3, HOXC6, CYP19A1, CD48, PSD3, APOA1, CYRIA, TLN2, CHST15, TASOR, PRMT2, KRT14, CALM1, AQP1, THRA, OLFM1, FAM111A, NDFIP1, TSPOAP1, EPS15, PDE2A, RPS6KA1, MTMR9, DHCR24, BCL11B, LCP1, HLA-F, PPP2R5A, TERF2IP, LYST, SRRM2, CLASP2, PTPRD, OGDHL, ASXL2, SEZ6L, SLCO2B1, FAM13B, TM7SF2, PCSK6, FOXJ3, CFD, VCAM1, LEP, KAT6A,

In [58]:
from graph_of_thoughts import controller, language_models, operations

from graph_of_thoughts.vector_db.weaviate import WeaviateClient
import logging
import dill
import time
with open("questions.pkl", "rb") as f:
    questions = dill.load(f)

# Problem input

operations_log = {}
# with open("operations_log_cot.pkl", "rb") as f:
#     operations_log = dill.load(f)
    


for json_file,values in questions.items():
    print(json_file)
    if json_file.startswith("MCQ"):
      question_type = "multiple choice"
    elif json_file.startswith("True_or_False"):
      question_type = "true/false"
    elif json_file.startswith("OpenEnded_1hop"):
      question_type = "list"
    elif json_file.startswith("OpenEnded_2hop"):
      question_type = "list"
    elif json_file.startswith("OpenEnded_genes"):
      question_type = "open ended"
    
    if json_file not in operations_log:
      operations_log[json_file] = {}
    for question in values:
        print('question:', question["question"])
        print('answer:', question["answer"])
        
        if question["question"] in operations_log[json_file]:
          # print("already done")
          continue

        def cot() -> operations.GraphOfOperations:
          """
          Generates the Graph of Operations for the IO method.

          :return: Graph of Operations
          :rtype: GraphOfOperations
          """
          operations_graph = operations.GraphOfOperations()

          operations_graph.append_operation(operations.Generate(1, 1))
          operations_graph.append_operation(operations.Score(1, False))
          # operations_graph.append_operation(operations.GroundTruth())

          return operations_graph
        # Retrieve the Graph of Operations
        io = cot()

        config_file = "config.json"
        config_file = "config_v1.json"

        # Configure the Language Model (Assumes config.json is in the current directory with OpenAI API key)
        lm = language_models.AzureGPT(config_file, model_name="azuregpt")
        vdb = WeaviateClient(config_file)
        
        # Create the Controller
        try:
          ctrl = controller.Controller(
            lm, 
            cot, 
            ALZKBPrompter(vdb,lm), 
            ALZKBParser(),
            # The following dictionary is used to configure the initial thought state
            {
              "question": question["question"],
              "ground_truth" : question["answer"],
              "question_type": question_type,
              "current": "",
              "phase": 0,
              "method": "cot"
              # "method" : "got"
            }
          )
          
          # ctrl.logger.setLevel(logging.DEBUG)

          # Run the Controller and generate the output graph
          ctrl.run()
        except Exception as e:
          print(e)
        # ctrl.output_graph("output_got.json")
        operations_log[json_file][question["question"]] = ctrl.graph.operations
        
        # delete the controller to free up memory
        del ctrl
        # time.sleep(1)
        #break
  #break
    



TypeError: 'dict_keys' object is not subscriptable

In [37]:
#show all the logs from ctrl logger which uses the logging module
import logging
logger = logging.getLogger("ctrl")
logger.setLevel(logging.DEBUG)
logger.debug("test")

# logger.handlers[0].stream.getvalue()


DEBUG:ctrl:test


<bound method Logger.info of <Logger ctrl (WARNING)>>

In [11]:
vdb.client.query.get(vdb.db, ["knowledge"]).with_near_vector({'vector': embedded_question}).with_additional(["distance"]).with_limit(2000).do()


# (get_client().query
#         .get(class_name, properties))
#         .with_near_vector(near_vector)
#         .with_limit(limit)
#         .with_additional(additional)
#         .do()


{'errors': [{'locations': [{'column': 36, 'line': 1}],
   'message': 'Argument "nearVector" has invalid value {vector: [[-0.007626513484865427, -0.0017940843245014548, 0.04350820183753967, -0.03055901639163494, -0.02252204716205597, 0.03702036663889885, -0.016868364065885544, -0.0012710855808109045, -0.029685145244002342, -0.0009293158072978258, 0.0003525276551954448, 0.030453091487288475, -0.0022790166549384594, 0.03463708236813545, -0.005726505536586046, 0.005984694696962833, 0.04072770103812218, 0.023859335109591484, -0.004756640642881393, 0.003248881548643112, -0.0012719130609184504, 0.031009191647171974, 0.031009191647171974, -0.01877499371767044, 0.017119934782385826, 0.009625825099647045, 0.01096311304718256, -0.04276673495769501, -0.007255780044943094, -0.012042212300002575, 0.029049601405858994, -0.013220613822340965, 4.688472836278379e-05, -0.017464186996221542, 0.016431428492069244, -0.003429282922297716, 0.0018288405844941735, -0.002863252768293023, -0.010665202513337135, 0